# Llama 3.1 8b 프롬프트, 하이퍼파라미터 튜닝

Hugging Face에서 Llama 3.1 8b 토큰 발급받아, Google Cloud Platform의 GPU 가상환경 위에서 실습하였습니다.

프롬프트와 하이퍼파라미터를 조금씩 수정해 가며 질문에 따른 답변을 확인했습니다.

프롬프트 엔지니어링으로는 따옴표와 기호 등을 이용하거나, 모형에게 역할을 지정(예: 너는 테니스 전문 AI 모형이야)하여, 특정 패턴에 대한 모형의 출력 변화를 확인했습니다.  

하이퍼파라미터 튜닝으로는 생성하는 문장의 다양성을 조절하는 temperature, 특정 단어 다음 나올 단어의 확률을 조정하는 top_p, 모형이 가지고 있는 자원 중 답변으로 생성할 최대 자원의 수를 결정하는 max_new_tokens을 조정해 봤습니다.

In [ ]:
import os 
import json

os.environ["HF_TOKEN"] = "(your huggingface token)" # set your huggingface token

In [ ]:
import torch
from transformers import pipeline

device = "cuda" if torch.cuda.is_available() else "cpu"

llama_31 = "meta-llama/Meta-Llama-3.1-8B-Instruct" 

generator = pipeline(model=llama_31, device=device, torch_dtype=torch.bfloat16)

In [ ]:
def generate_response(system_prompt, user_prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    
    generation = generator(
        messages,
        do_sample=False,
        temperature=1.0,
        top_p=30,
        max_new_tokens=500
    )
    
    return generation[0]['generated_text']

In [ ]:
user_prompt = "text:한글 -> eng  는  한글을 영어로 번역하고 그 번역된 내용의 답변 해달란 의미야. \
text: 머신러닝과 딥러닝의 장단점을 알려줘. -> eng "

In [ ]:
response = generate_response("You are a AI assistant who always responds in korean", user_prompt)
ai_message = json.dumps(response, ensure_ascii=False, indent=3)

print(f" Llama3.1 > {ai_message} \n")

In [ ]:
history = []
history = response

In [ ]:
history.append({"role": "user", "content": "```다음 문장을 영어로 번역해줘. ```머신러닝 이란 무엇입니까?"})

history


In [ ]:
generation = generator(
    history,
    do_sample=False,
    temperature=0.2,
    top_p=60,
    max_new_tokens=100
)

generation[0]['generated_text']

In [ ]:
def chat_with_user(input_text, chat_history):
    
    if len(chat_history) == 0:
        chat_history.append({"role": "system", "content": "You are a AI assistant who always responds in korean"})

    chat_history.append({"role": "user", "content": input_text})

    generation = generator(
        chat_history,
        do_sample=False,
        temperature=1.0,
        top_p=1,
        max_new_tokens=300
    )

    ai_response = generation[0]['generated_text']
    
    return ai_response

In [ ]:
chat_history = []
while True:
    user_prompt = input("USER > ")
    if user_prompt.lower() == "quit":
        break
        
    response = chat_with_user(user_prompt, chat_history)
    chat_history = response
    ai_message = json.dumps(response, ensure_ascii=False, indent=3)
    print(f" Llama3.1 > {ai_message} \n")

In [ ]:
user_prompt = "You are one of the best bilingual AI assistants for tennis coaching. \
You can comprehend both English and Korean. Please answer as Korean.\
테니스 기술을 향상시키는 방법에 대한 일반적인 지식과 전문 지식을 바탕으로, 테니스 기술을 향상시키는 데 필요한 구체적인 지시사항을 제공하는 방법을 알려줘. \
다양한 관점과 시나리오를 고려하여, 테니스 기술을 향상시키는 방법에 대한 명확하고 구체적인 예시와 셈플 데이터를 포함해야 해. \
편향과 민감한 내용에 대한 주의사항을 포함하여, 테니스 기술을 향상시키는 방법에 대한 구체적인 기준이나 체크리스트를 제공해야 해"

In [ ]:
response = generate_response("You are a AI assistant who always responds in korean", user_prompt)
ai_message = json.dumps(response, ensure_ascii=False, indent=3)

print(f" Llama3.1 > {ai_message} \n")

In [ ]:
system_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|> \
You are a helpful assistant with tool calling capabilities. When you receive a tool call response, use the output to format an answer to the orginal use question.<|eot_id|><|start_header_id|>user<|end_header_id|>\
Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.\
Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}. Do not use variables.\
{\
    "type": "function",\
    "function": {\
    "name": "get_current_conditions",\
    "description": "Get the current weather conditions for a specific location",\
    "parameters": {\
        "type": "object",\
        "properties": {\
        "location": {\
            "type": "string",\
            "description": "The city and state, e.g., San Francisco, CA"\
        },\
        "unit": {\
            "type": "string",\
            "enum": ["Celsius", "Fahrenheit"],\
            "description": "The temperature unit to use. Infer this from the user's location."\
        }\
        },\
        "required": ["location", "unit"]\
    }\
    }\
}\
\
Question: what is the weather like in San Fransisco?<|eot_id|>>"""

user_prompt = ""

In [ ]:
response = generate_response(system_prompt, user_prompt)
ai_message = json.dumps(response, ensure_ascii=False, indent=3)

print(f" Llama3.1 > {ai_message} \n")